<a href="https://colab.research.google.com/github/adityamavle/Domain_Specific_Financial_LLM/blob/master/llama2_finance_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

###llama2-finance-alpaca inference

In [4]:
device_map = {"": 0}
base_model = AutoModelForCausalLM.from_pretrained(
    'adityamavle/llama-2-7b-finance-alpaca-v2',
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
# model = PeftModel.from_pretrained(base_model, new_model)
# model = model.merge_and_unload()
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained('NousResearch/Llama-2-7b-chat-hf', trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [5]:
def generate_text(model_name, tokenizer, prompt):
    logging.set_verbosity(logging.CRITICAL)
    # Initialize the pipeline with the provided model and tokenizer
    pipe = pipeline(task="text-generation", model=model_name, tokenizer=tokenizer, max_length=1024)
    # Run the pipeline with the formatted prompt
    formatted_prompt = f"<s>[INST] {prompt} [/INST]"
    result = pipe(formatted_prompt)
    # Print the prompt and the generated text
    print("This is the prompt")
    print(result[0]['generated_text'])

In [4]:
# Run text generation pipeline with our next model
prompt = "Give a list of 3 things that present the most financial risk to mid-size technological companies"
pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=1024)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print("This is the prompt")
print(result[0]['generated_text'])

This is the prompt
<s>[INST] Give a list of 3 things that present the most financial risk to mid-size technological companies [/INST] Here are three things that present the most financial risk to mid-size technological companies: 1. (Lack of) Diversification: Mid-size companies often have a narrow focus on a particular product or service. This can make them vulnerable to changes in the market or economy. For example, if a mid-size company is heavily invested in a particular technology, and that technology becomes obsolete, the company may struggle to adapt. 2. (Over) Investment in Research and Development: While research and development (R&D) is important for staying competitive, it can also be a significant financial risk. If a mid-size company invests too much in R&D, it may not be able to generate enough revenue to cover the costs. This can lead to a vicious cycle of increased spending and decreased profits. 3. (Lack of) Cash Flow Management: Mid-size companies often have limited re

In [6]:
prompt = "Analyze the sentiment of the following sentence and answer in one word as 'positive', 'negative' or 'neutral': The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model"
generate_text(base_model, tokenizer,prompt)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


This is the prompt
<s>[INST] Analyze the sentiment of the following sentence and answer in one word as 'positive', 'negative' or 'neutral': The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model [/INST] Positive


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch

data = pd.read_csv('/content/data.csv')
data = data.head(100)

In [9]:
y

NameError: name 'y' is not defined

In [26]:
data['Sentence']

0     The GeoSolutions technology will leverage Bene...
1     $ESI on lows, down $1.50 to $2.50 BK a real po...
2     For the last quarter of 2010 , Componenta 's n...
3     According to the Finnish-Russian Chamber of Co...
4     The Swedish buyout firm has sold its remaining...
                            ...                        
95    M-Real said there are ` no grounds ' for the r...
96    Lember said the matter was topical also in Est...
97    - Moody 's said it gave P A1 long-term senior ...
98                                         $TSLA recall
99    The liquidity providing was interrupted on May...
Name: Sentence, Length: 100, dtype: object

In [30]:
X = data["Sentence"]
y = data["Sentiment"]

# Function to evaluate LM accuracy
def evaluate_lm_accuracy(lm, X_data, y_true):
    y_pred = []
    pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=1024)
    for prompt in X_data:
      prompt = f"Analyze the financial sentiment of the sentences, response will be one of the 3 labels = positive,negative,neutral: {prompt}"
      #print(prompt)
      ############inference code###########################
      result = pipe(f"<s>[INST] {prompt} [/INST]")
      parts = result[0]['generated_text'].split("[/INST] ")
      answer = parts[1].split(".")[0]
      answer = answer.lower()
      sentiments = ["positive", "negative", "neutral"]
      ans = None
      for word in answer.lower().split():
          if word.strip(",.?!").lower() in sentiments:
              ans = word.strip(",.?!").lower()
          else:
              ans = 'neutral'
              break
      y_pred.append(ans)
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

metrics_lm1 = evaluate_lm_accuracy(base_model, X, y)
#metrics_lm2 = evaluate_lm_accuracy(lm_model_2, X, y)

print(f"Metrics of model 1: {metrics_lm1}")
#print(f"Metrics of model 2: {metrics_lm2}")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Metrics of model 1: 0.53


In [37]:
# del base_model
# del pipe
# import gc
gc.collect()
gc.collect()

0

In [ ]:
This is awesome! // Negative
This is bad! // Positive
Wow that movie was rad! // Positive
What a horrible show! //



###llama2-base-inference


In [38]:
base_model = AutoModelForCausalLM.from_pretrained(
    'NousResearch/Llama-2-7b-chat-hf',
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

In [42]:
gc.collect()

632

In [ ]:
metrics_lm_base = evaluate_lm_accuracy(base_model, X, y)
print("Accuracy Metrics of LLaMA-base model are",metrics_lm_base)